# Lab 11: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [2]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

2024-04-30 16:37:47,253 | INFO     |: Logger demo_notebook.log initialized.


In [3]:
# Start the plotter
START_PLOTTER()

# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [4]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

        # Arrays to store sensor readings
        self.tof_reading = [] # stores distance in meters
        self.yaw = []

    def notification_handler(self, uuid, byte_array):
        self.yaw.append(-((float) (self.ble.bytearray_to_string(byte_array).split(", ")[0])))
        self.tof_reading.append(((float) (self.ble.bytearray_to_string(byte_array).split(", ")[2]))*0.001)
    
    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        raise NotImplementedError("get_pose is not implemented")

    def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        # raise NotImplementedError("perform_observation_loop is not implemented")

        # Send commands via Bluetooth
        self.ble.start_notify(ble.uuid['RX_STRING'], self.notification_handler)
        self.ble.send_command(CMD.KP_SET, "5")
        self.ble.send_command(CMD.KI_SET, "0.01")
        self.ble.send_command(CMD.KD_SET, "500")

        self.yaw.clear()
        self.tof_reading.clear()
        self.ble.send_command(CMD.START_MAPPING, "")
        print("Starting observation...")
        
        # Wait for the robot to complete a 360-degree observation
        asyncio.run(asyncio.sleep(20))

        self.ble.send_command(CMD.GET_DATA_MAP, "")
        print("Obtaining sensor data...")
        asyncio.run(asyncio.sleep(5))

        # Clockwise data -> Counter-clockwise data
        self.tof_reading = [self.tof_reading[0]] + self.tof_reading[-2:0:-1]
        self.yaw = [self.yaw[0]] + self.yaw[-2:0:-1]

        # Convert into numpy column arrays
        sensor_ranges = np.array(self.tof_reading)[np.newaxis].T
        sensor_bearings = np.array(self.yaw)[np.newaxis].T
        
        return sensor_ranges, sensor_bearings

In [21]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2024-04-30 16:48:03,131 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:89:c5:6d:2f:4b
2024-04-30 16:48:07,179 | INFO     |: Connected to c0:89:c5:6d:2f:4b


In [6]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2024-04-30 16:38:01,472 | INFO     |:  | Number of observations per grid cell: 18
2024-04-30 16:38:01,481 | INFO     |:  | Precaching Views...


C:\Users\Kaiyuan\Documents\Cornell\Courses\Fast_Robots\Project\ble_python\lab11_tasks\localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2024-04-30 16:38:09,463 | INFO     |:  | Precaching Time: 7.982 secs
2024-04-30 16:38:09,471 | INFO     |: Initializing beliefs with a Uniform Distribution
2024-04-30 16:38:09,471 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run an update step of the Bayes Filter

In [22]:
# Reset Plots
cmdr.reset_plotter()

# current_gt = [-0.91, -0.6]
# current_gt = [1.52, -0.91]
current_gt = [1.52, 0.91]
cmdr.plot_gt(current_gt[0], current_gt[1])

In [23]:
# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# # Plot Odom and GT
# # current_odom, current_gt = robot.get_pose()
# current_gt = [1.65, 1]
# cmdr.plot_gt(current_gt[0], current_gt[1])
# # cmdr.plot_odom(current_odom[0], current_odom[1])

2024-04-30 16:48:30,297 | INFO     |: Initializing beliefs with a Uniform Distribution
2024-04-30 16:48:30,307 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
Starting observation...
Obtaining sensor data...
2024-04-30 16:48:56,103 | INFO     |: Update Step
2024-04-30 16:48:56,109 | INFO     |:      | Update Time: 0.006 secs
2024-04-30 16:48:56,114 | INFO     |: Bel index     : (11, 6, 8) with prob = 1.0
2024-04-30 16:48:56,116 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2024-04-30 16:48:56,117 | INFO     |: Belief        : (1.829, 0.610, -10.000)
2024-04-30 16:49:18,017 | INFO     |: Disconnected from C0:89:C5:6D:2F:4B
